In [1]:
%load_ext autoreload
#%reload_ext autoreload

%autoreload 2

import sys
import logging

sys.path.append("../model")

import os
import pandas as pd
import mlflow as mlf
from tqdm.auto import tqdm

tqdm.pandas()

from pysentimiento.preprocessing import preprocess_tweet
from analyzer_blstm import (
    AnalyzerForSequenceClassification,
    create_analyzer_blstm,
)

logging.getLogger("Utils").setLevel(logging.DEBUG)


ROBERTUITO_RUN_ID = "52089d0757e64bf588f2c75e439ae4e0"  # FINAL

ROBERTUITO_BLSTM_RUN_ID = "631eeac0d2d84a24a6b41a7ff6cc3ba4"  # FINAL

MIN_TEXT_LENGTH = 4

In [2]:
model_analyzer = create_analyzer_blstm()

DEBUG:Utils:Loading model pysentimiento/robertuito-base-uncased
DEBUG:Utils:Using device CUDA


In [3]:
runs_path = os.path.abspath(os.path.join("..", "model", "mlruns"))
mlf.set_tracking_uri("file:/ " + runs_path)

columns = [
    "run_id",
    "status",
    "params.lstm_hidden_dim",
    "params.lstm_num_layers",
    "metrics.train_runtime",
    "metrics.eval_macro_f1",
    "metrics.train_loss",
    "metrics.eval_macro_recall",
    "metrics.eval_macro_precision",
    "artifact_uri",
]

runs = mlf.search_runs(
    filter_string='status="FINISHED"', order_by=["metrics.eval_macro_f1 DESC"]
)[columns]

## Analysis tweets


In [4]:
def process_analysis_x(
    row: pd.Series, model_analyzer: AnalyzerForSequenceClassification
):

    text = row["text"]

    if not text or len(text) < MIN_TEXT_LENGTH:
        return {
            "text": text,
            "analysis": None,
        }

    text_analysis = model_analyzer.predict(text).probas

    max_proba = max(text_analysis.items(), key=lambda x: x[1])[0]

    return {
        "text": text,
        "analysis": {
            "max_proba": max_proba,
            "probas": text_analysis,
        },
    }

In [5]:
df_x = pd.read_excel("../data/append_tweets.xlsx")
df_x["text"] = df_x["text"].astype(str)

In [10]:
analysis = df_x.progress_apply(
    lambda row: process_analysis_x(row, model_analyzer), axis=1
)

df_x["analysis"] = analysis
df_x.to_excel("../data/append_tweets_analysis.xlsx", index=False)

  0%|          | 0/1323 [00:00<?, ?it/s]

## Analysis tiktoks


In [7]:
def process_analysis_tiktok(
    row: pd.Series, model_analyzer: AnalyzerForSequenceClassification
):

    description_text = row["description"]

    comments_text = row["comments_text"].split("|")

    comments_text = [
        comment for comment in comments_text if len(comment) >= MIN_TEXT_LENGTH
    ]

    description_analysis = None

    if not description_text or len(description_text) < MIN_TEXT_LENGTH:
        probas_desc = model_analyzer.predict(description_text).probas

        if len(probas_desc) == 0:
            probas_desc = None
        else:
            max_proba_description = max(probas_desc.items(), key=lambda x: x[1])[0]
            description_analysis = {
                "sentiment": max_proba_description,
                "probas": probas_desc,
            }
    else:
        description_analysis = None

    comments_analysis = []

    for comment_text in tqdm(
        comments_text, desc="Analyzing comments", leave=False, unit="comment"
    ):
        if len(comment_text) < MIN_TEXT_LENGTH:
            comments_analysis.append({"text": comment_text, "analysis": None})

        probas = model_analyzer.predict(comment_text).probas

        if len(probas) == 0:
            comments_analysis.append(
                {
                    "text": comment_text,
                    "analysis": None,
                }
            )
        else:
            max_proba = max(probas.items(), key=lambda x: x[1])[0]

            comments_analysis.append(
                {
                    "text": comment_text,
                    "analysis": {
                        "sentiment": max_proba,
                        "probas": probas if len(probas) > 0 else None,
                    },
                }
            )

    return {
        "description": {
            "text": description_text,
            "analysis": description_analysis,
        },
        "comments": comments_analysis,
    }

In [8]:
df_tiktok = pd.read_excel("../data/append_tik_tok.xlsx")
df_tiktok["description"] = df_tiktok["description"].astype(str)
df_tiktok["comments_text"] = df_tiktok["comments_text"].astype(str)

In [9]:
analysis = df_tiktok.progress_apply(
    lambda row: process_analysis_tiktok(row, model_analyzer), axis=1
)

df_tiktok["analysis"] = analysis
df_tiktok.to_excel("../data/append_tik_tok_analysis.xlsx", index=False)

  0%|          | 0/1057 [00:00<?, ?it/s]

Analyzing comments:   0%|          | 0/2 [00:00<?, ?comment/s]

Analyzing comments:   0%|          | 0/40 [00:00<?, ?comment/s]

Analyzing comments:   0%|          | 0/8 [00:00<?, ?comment/s]

Analyzing comments:   0%|          | 0/29 [00:00<?, ?comment/s]

...

Analyzing comments:   0%|          | 0/5 [00:00<?, ?comment/s]

Analyzing comments:   0%|          | 0/20 [00:00<?, ?comment/s]

Analyzing comments:   0%|          | 0/19 [00:00<?, ?comment/s]

Analyzing comments:   0%|          | 0/14 [00:00<?, ?comment/s]

Analyzing comments:   0%|          | 0/1 [00:00<?, ?comment/s]